# Identify relevant law articles for each case

In [35]:
import yaml

with open("../../src/config/cfg.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

## Validate schema of received json

In [36]:
import json

f = open('../../data/01_raw/dev/sachverhalt.json')
data = json.load(f)

print(data)

{'beteiligter': 'Sample Name', 'fahrzeug': 'Sample Vehicle', 'aktionen': [{'id': 1, 'beschreibung': 'Sample Description'}, {'id': 2, 'beschreibung': 'Sample Description 2'}]}


In [37]:
import sys
# add path to sys.path in order to access schemas
sys.path.append('../../src/schemas')

from sachverhalt import schema_sachverhalt

schema_sachverhalt.validate(data)

## Check case
for each aktion in sachverhalt
* vectorize aktion
* search best matching law article for aktion
* add id to aktion (attr: artikel_id)

### setup pinecone connection

In [38]:
import os
import pinecone

api_key = os.environ["PINECONE_API_KEY"]
pinecone.init(api_key=api_key, environment=config['vectorization']['environment'])

index_name = "law"
index = pinecone.Index(index_name)

### load vectorstore

In [39]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

openai_api_key = os.environ.get('OPENAI_API_KEY')
model_name = config['sentence_transformer']['model_name']

embed = HuggingFaceEmbeddings(
    model_name=model_name
)
print(embed)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
) model_name='all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={}


In [40]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_key=text_field
)

In [42]:
query = "Missachtung des Vortritts und Kollision mit rechter Fahrzeugseite von G"

vectorstore.similarity_search(
    query,  # our search query
    k=10  # return 10 most relevant docs
)

[Document(page_content='Die Zeichengebung entbindet den Fahrzeugführer nicht von der gebotenen Vorsicht. ', metadata={}),
 Document(page_content='Der Fahrzeugführer hat beim Hintereinanderfahren einen ausreichenden Abstand zu wahren, so dass er auch bei überraschendem Bremsen des voranfahrenden Fahrzeugs rechtzeitig halten kann.', metadata={}),
 Document(page_content='Auf Kreisverkehrsplätzen können Radfahrer vom Gebot des Rechtsfahrens abweichen', metadata={}),
 Document(page_content='An ausserorts abgestellten mehrspurigen Fahrzeugen sind die Standlichter oder die Parklichter auf der Seite des Verkehrs zu verwenden. Bei Fahrzeugen ohne derartige Lichter sind die für die entsprechende Fahrzeugart vorgeschriebenen Lichter zu verwenden.', metadata={}),
 Document(page_content='Reiter sowie Führer von Pferden und anderen grösseren Tieren sind den Fahrzeugführern beim Vortritt gleichgestellt.', metadata={}),
 Document(page_content='Für die Benützer von fahrzeugähnlichen Geräten gelten die 